# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [1]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [73]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
if api_key:
    print("True")  # Verify the key is loaded


True


In [12]:
ej="""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""

In [ ]:
## bloque conexion a Cohere
import cohere
co = cohere.ClientV2()
# alternativa:
# co = cohere.ClientV2(api_key)


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [68]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

{'paciente': {'nombre': 'María González', 'edad': 45},
 'fecha_admision': '2023-08-05',
 'sintomas': ['fatiga crónica', 'dolores musculares'],
 'diagnostico': 'fibromialgia',
 'tratamiento': ['fisioterapia', 'medicamentos analgésicos']}

In [70]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [71]:
co = cohere.ClientV2()
response = co.chat(
    model= "command-r-plus-08-2024",
    messages=[{"role": "system", "content": "Sos un asistente capaza de interpretar entidades en un texto medico. Extrae el nombre y edad del paciente, la fecha de admision, los sintomas,"
                "el diagnotico y  el tratamiento recomendado. Los sintomas deben estar por separado. Si hay mas de un tratamiento debe estar por separado. La fecha transformarla a formato dd/mm/aaaa." 
                "Si alguna de estas entidades no se encuentran en el texto no inventes informacion.  Dar la respuesta en formato JSON"},
              {"role": "user", "content": f"{text_to_analize}"}],
)

llm_response=response.message.content[0].text
print(llm_response)



{
  "paciente": {
    "nombre": "Sofía López",
    "edad": "28 años"
  },
  "fecha_admision": "03/04/2023",
  "sintomas": [
    "Fiebre alta",
    "Tos persistente"
  ],
  "diagnostico": "Neumonía",
  "tratamiento": [
    "Antibióticos",
    "Reposo"
  ]
}


In [72]:
# test

final_result = json.loads(llm_response)
final_result

{'paciente': {'nombre': 'Sofía López', 'edad': '28 años'},
 'fecha_admision': '03/04/2023',
 'sintomas': ['Fiebre alta', 'Tos persistente'],
 'diagnostico': 'Neumonía',
 'tratamiento': ['Antibióticos', 'Reposo']}

## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [10]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."
    


In [11]:
contacts = {
                        'Joaquin Lopez':{'phone': 15456663258, 'email': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'phone': 1545554178, 'email': 'FOncativo@hotmail.com'}
}

In [139]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

In [12]:
# definicion del esquema de herramientas 


function_map={"add_contact":add_contact,
              "get_information": get_information}

tools = [
    {
        "type": "function",
        "function": {  
            "name": "add_contact",
            "description": "Add contact to dict and returns a message confirming the addition of the contact",
            "parameters": {  
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string", 
                        "description": "Name of new contact"
                    },
                    "phone": {
                        "type": "string",
                        "description": "phone number of new contact"
                    },
                    "email": {
                        "type": "string",
                        "description": "email of new contact"
                    }
                },
                "required": ["name", "phone", "email"]  #
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "get information about of a contact. If request ",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of contact"
                    }
                },
                "required": ["name"]
            }
        }
    }
]

In [17]:
# Definicion de promt y mensaje al modelo
system_promt=f"""Sos un asistente de agenda capaza de interpretar los pedidos. Tenes a dispocicion estas tools: {tools}
y este es el diccionario de contactos que tenes que usar {contacts}. Solo responde a los pedidos de agregar un contacto o consultas sobre la información de algun contacto que se encuentre en el diccionario brindado. Si el usuario pide el telefono solo brindarle el campo "phone", si pide el mail o email dar el campo "mail" y si no especifica brindar ambos. """

messages=[{"role": "system", "content": f"{system_promt}"},
              {"role": "user", "content": "dame e numero de Joaquin Lopez"}]

# llamada al modelo con la solicitud del usuario
co = cohere.ClientV2()
response = co.chat(
    model="command-a-03-2025",
    messages=messages,
    tools=tools, 
)

    
# busqueda de la funcion a utilizar (si la necesita) y guardado de la salida en tool_content
if response.message.tool_calls:
    messages.append(response.message)
    for tc in response.message.tool_calls:

        tool_result = function_map[tc.function.name](
            **json.loads(tc.function.arguments)
        )
      
        tool_content = []
        for data in tool_result:
            tool_content.append(
                {
                    "type": "document",
                    "document": {"data": json.dumps(data)},
                }
            )
       
       # agrega la información recuperada de la herramienta
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tc.id,
                "content": tool_content,
            }
        )

# Generacion de respuesta con la informacion recuperada

response = co.chat(
    model="command-a-03-2025", messages=messages, tools=tools
)
print(response.message.content[0].text)

El número de Joaquin Lopez es 15456663258.


In [15]:
contacts

{'Joaquin Lopez': {'phone': 15456663258, 'email': 'Joacolocolopez@gmail.com'},
 'Flavio Oncativo': {'phone': 1545554178, 'email': 'FOncativo@hotmail.com'},
 'Juan Pérez': {'phone': '555-1234', 'email': 'juanperez@mail.com'}}

## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [173]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [67]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""



In [41]:
   
def language_detect(pregunta,respuesta):
    
    """ Traduce una respuesta con un modelo de traduccion
    Argumentos:
        pregunta (str): entrada del usuario al LLM
        respuesta (str): respuesta del primer LLM en español
    Retorna:
        respuesta_trad (str): respuesta traducida por el LLM .          
    
    ----NOTA----: 
        Falta optimizar esta funcion, ya que si el lenguaje es español no debería pasar por este modelo.
    """


    # Definicion del promt para el modelo de traduccion 
    system_promt_DL= f"""Sos un asistente con capacidad de interpretar y traducir cuando sea necesario. Tu funcion es responder siempre en el mismo idioma de la pregunta.
    Estilo de respuesta: 
    - Solo dar la traduccion de {respuesta}
    - Mantener los emojis que resumen el contenido y la expresion "Hakuna Matata!" al finalizar la oracion
    
 
    Pregunta del usuario: {pregunta}"""
    
    
    # Llamada al modelo para traducir. 
    #       seed=1212, fija la semilla que normalmente es aleatoria y permite resultados repetibles
    #       temperature=0, fuerza al modelo a ser exacto y repetitivo
    #       k=0, valor que controla la cantidad de tokens probables, se queda solo con el mas probable
    #       p=0, valor que limita al modelo para eligir solo el token mas probable
    co = cohere.ClientV2()
    response = co.chat(
        model= "command-a-translate-08-2025",
        messages= [{"role": "system", "content": f"{system_promt_DL}"},
                   {"role": "user", "content": f"{pregunta}"}],
        temperature=0, 
        seed=1212,
        k=0,
        p=0
    )
    
    # Obtiene la respuesta
    respuesta_trad= response.message.content[0].text
      
    return respuesta_trad

In [63]:
# Definicion del promt para el modelo que genera la respuesta

system_promt= f""" Eres un asistente con capacidad de interpretar el contexto brindado. Tu funcion es responder solo sobre el contenido del contexto dado, si la pregunta no tiene relacion con el contexto la respuesta debe ser "Lo siento, no puedo ayudarte con eso". 
Idioma:
Responde en el idioma de entrada al modelo 
Estilo de respuesta:
- Extension maxima: 1 oración 
- Responde siempre en tercera persona. 
- Utiliza la expresion "Hakuna Matata!" al final de todas las respuestas.


 Contexto: {historia} 
"""


def history_answer(pregunta):
    """ Responde a una pregunta del contexto siguiendo las especificaciones del promt 
    Argumentos:
        pregunta (str): entrada del usuario al LLM
    Retorna: 
        respuesta_al_usuario (str): respuesta traducida 

    NOTA:
        Una opcion para optimizar la obtención de resultados iguales ante la misma pregunta, es utilizar un archivo JSON de forma local que guarde la pregunta y su respuesta, o bien guardarlas en la memoria cache. El problema de estas practicas radica en que la pregunta debe ser exactamente igual para que esto funcione, sino se le vuelve a consultar al modelo. Para el fin de este ejercicio, y por el tiempo con el que contaba, decidi que el uso de los parametros semilla, temperatura, top-k y top-p para forzar la respuesta era lo mas adecuado. En las pruebas realizadas a veces la respuesta varía ligeramente, otras veces es la misma. 
    
        """

    # 
    #Llamada al LLM para responder la pregunta
    #       seed=1212, fija la semilla que normalmente es aleatoria y permite resultados repetibles
    #       temperature=0, fuerza al modelo a ser exacto y repetitivo
    #       k=0, valor que controla la cantidad de tokens probables, se queda solo con el mas probable
    #       p=0, valor que limita al modelo para eligir solo el token mas probable
    co = cohere.ClientV2()
    response = co.chat(
    model="command-a-03-2025",
    messages=[{"role": "system", "content": f"{system_promt}"},
         {"role": "user", "content": f"{pregunta}"}],
    temperature=0, 
    seed=1212,
    k=0,
    p=0 
    )
       
    # Manda la respuesta a la funcion para detectar el lenguaje y traducir la respuesta      
    respuesta_al_usuario=language_detect(pregunta, response.message.content[0].text)
    
    return respuesta_al_usuario


In [66]:
pregunta="""que le paso a thomas?"""

print(history_answer(pregunta))

Thomas fue reclutado para la guerra, luchó en una batalla caótica, presenció la muerte y regresó marcado por la violencia, perdiendo su inocencia. 😢😔 Hakuna Matata!


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [36]:
from collections import deque

# Definicion del promt y mensaje para el sistema
system_promt= """ Eres un asistente personal. Tu objetivo es responder con consejos utiles como si fueras un tutor. Mantene un tono entusiasta y responde siempre de manera positiva. 
Idioma:
respondes exclusivamente en español castellano rioplatense.
Estilo de respuesta:
- extension maxima: 70 tokens
- nunca inventes informacion
"""
system_msg= [{"role": "system", "content": f"{system_promt}"}]

# variable que contendra hasta 3 preguntas con sus respuestas
chat_history=deque(maxlen=6)   


# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()     

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    global chat_history
    
    
    user = {"role": "user", "content": f"{message}"}
    chat_history.append(user)
    
    messages= system_msg + list(chat_history)
    
    # Conexion al modelo
    co = cohere.ClientV2()
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=messages,
        max_tokens=70,
        temperature=0.5)
 
 # guarda la pregunta y respuesta en el historial
    chat_history.append(response.message)
        
    rta_usuario= response.message.content[0].text
     
    return (rta_usuario)

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)


Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [38]:
for i in range(len(chat_history)):
    print(chat_history[i])

{'role': 'user', 'content': 'que puedo hacer en cordoba?'}
role='assistant' tool_calls=None tool_plan=None content=[TextAssistantMessageResponseContentItem(type='text', text='¡Córdoba es una ciudad increíble con muchas opciones para disfrutar! Aquí te doy algunas ideas de actividades que podés hacer:\n\n- Visitar el Centro Histórico: Caminar por las calles del centro es una experiencia única. Podés explorar la Catedral, el Cabildo, la Manzana Jesuítica (declarada Patrimonio de la Humanidad')] citations=None
{'role': 'user', 'content': 'may museos?'}
role='assistant' tool_calls=None tool_plan=None content=[TextAssistantMessageResponseContentItem(type='text', text='¡Sí, Córdoba tiene una gran oferta de museos para visitar! Te menciono algunos de los más destacados:\n\n- Museo Superior de Bellas Artes Evita: Ubicado en el Palacio Ferreyra, este museo exhibe una colección de arte argentino y europeo.\n- Museo Histórico de la Universidad Nacional de Córdoba: Aquí podés conocer la historia d

### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.